Asynchronous class 23: April 21, 2021
============================

Time to do some nonlinear simulations of convection.  We'll stick here to Rayleigh-Benard convection.

# Homework
No written homework.  You're reading two things this week:
1. Chodhuri chapter 9 ("Rotation and hydrodynamics")
2. Chandrasekhar chapter 2 ("The effect of rotation")

You also have an exam due Friday; we've extended the deadline to mid-day Saturday April 24.

# Lecture

1. Watch [Intuition for nonlinear convection](https://youtu.be/LqJ8eWhJm7Q)

2. Work through the material below.

This notebook will require `dedalus` to complete, but the initial coding of the dedalus problem is done for you.  On the JupyterLab, I have files for everyone under folder `AS23`.  You need to move your file into your local home directory to run it successfully, based on experiences by students in the class on other exercises.  If you do this on RC's JupyterLab, download the notebook after you complete it (right click on it in the file tree to get that option), and upload that to Canvas.

# Nonlinear Rayleigh-Benard convection
The nonlinear Rayleigh-Benard equations, non-dimensionalized on a free-fall timescale are:
\begin{align}
\vec{\nabla}\cdot\vec{u} &= 0 \\
\frac{\partial}{\partial t} T + -\mathcal{P} \nabla^2 T  &= - \vec{u}\cdot\vec{\nabla}T\\
\frac{\partial}{\partial t} \vec{u} -\vec{\nabla}\varpi - T \vec{\hat{z}} - \mathcal{R} \nabla^2 \vec{u} &= - \vec{u}\cdot\vec{\nabla}\vec{u}
\end{align}
where the control parameters $\mathcal{R}$ and $\mathcal{P}$ are related to the Rayleigh number $\mathrm{Ra}$ and Prandtl number $\mathrm{Pr}$ by
\begin{align}
    \mathcal{R}^2 = \frac{\mathrm{Ra}}{\mathrm{Pr}} \\
    \mathcal{P}^2 = \mathrm{Ra}\mathrm{Pr}
\end{align}
as we derived earlier in the course.

A `dedalus` script that solves for 2-dimensional Rayleigh-Benard appears below.  Problems like this can be simple enough to solve on a single processor (like on RC's JupyterLab), though they can also require parallel computation for extreme parameters and high resolutions.

In [ ]:
from mpi4py import MPI
import time
import sys
import os
import numpy as np
import pathlib
import logging

from dedalus import public as de
from dedalus.extras import flow_tools
from dedalus.tools  import post


def rayleigh_benard(Rayleigh, Prandtl, nz, 
                    run_time=100, report_cadence=10, aspect=4):
    nx = aspect*nz
    case_name = data_dir = 'rayleigh_benard_Ra{:.1e}_Pr{:}_nx{:d}_nz{:d}'.format(Rayleigh,Prandtl, nx, nz)+'/'
    
    from dedalus.tools.config import config
    config['logging']['filename'] = os.path.join(data_dir,'logs/dedalus_log')
    config['logging']['file_level'] = 'DEBUG'

    import mpi4py.MPI
    if mpi4py.MPI.COMM_WORLD.rank == 0:
        if not os.path.exists('{:s}/'.format(data_dir)):
            os.mkdir('{:s}/'.format(data_dir))
        logdir = os.path.join(data_dir,'logs')
        if not os.path.exists(logdir):
            os.mkdir(logdir)
    # Parameters
    Lx, Lz = (aspect, 1.)

    # Create bases and domain
    x_basis = de.Fourier(  'x', nx, interval=(0, Lx), dealias=3/2)
    z_basis = de.Chebyshev('z', nz, interval=(-Lz/2, Lz/2), dealias=3/2)
    domain = de.Domain([x_basis, z_basis], grid_dtype=np.float64)

    # 2D Boussinesq hydrodynamics
    problem = de.IVP(domain, variables=['p','b','u','w','bz','uz','wz'])
    problem.meta['p','b','u','w']['z']['dirichlet'] = True
    problem.parameters['P'] = (Rayleigh * Prandtl)**(-1/2)
    problem.parameters['R'] = (Rayleigh / Prandtl)**(-1/2)
    problem.parameters['F'] = F = 1
    problem.add_equation("dx(u) + wz = 0")
    problem.add_equation("dt(b) - P*(dx(dx(b)) + dz(bz)) - F*w       = -(u*dx(b) + w*bz)")
    problem.add_equation("dt(u) - R*(dx(dx(u)) + dz(uz)) + dx(p)     = -(u*dx(u) + w*uz)")
    problem.add_equation("dt(w) - R*(dx(dx(w)) + dz(wz)) + dz(p) - b = -(u*dx(w) + w*wz)")
    problem.add_equation("bz - dz(b) = 0")
    problem.add_equation("uz - dz(u) = 0")
    problem.add_equation("wz - dz(w) = 0")
    problem.add_bc("left(b) = 0")
    problem.add_bc("left(u) = 0")
    problem.add_bc("left(w) = 0")
    problem.add_bc("right(b) = 0")
    problem.add_bc("right(u) = 0")
    problem.add_bc("right(w) = 0", condition="(nx != 0)")
    problem.add_bc("right(p) = 0", condition="(nx == 0)")

    logger = logging.getLogger(__name__)

    # Build solver
    solver = problem.build_solver(de.timesteppers.RK222)
    logger.info('Solver built')

    # Initial conditions
    x = domain.grid(0)
    z = domain.grid(1)
    b = solver.state['b']
    bz = solver.state['bz']

    # Random perturbations, initialized globally for same results in parallel
    gshape = domain.dist.grid_layout.global_shape(scales=1)
    slices = domain.dist.grid_layout.slices(scales=1)
    rand = np.random.RandomState(seed=42)
    noise = rand.standard_normal(gshape)[slices]

    # Linear background + perturbations damped at walls
    zb, zt = z_basis.interval
    pert =  1e-3 * noise * (zt - z) * (z - zb)
    b['g'] = F * pert
    b.differentiate('z', out=bz)

    # Initial timestep
    dt = 0.125

    # Integration parameters
    solver.stop_sim_time = run_time
    solver.stop_wall_time = np.inf
    solver.stop_iteration = np.inf

    # Analysis
    snapshots = solver.evaluator.add_file_handler(data_dir+'snapshots', sim_dt=0.25, max_writes=50, mode='overwrite')
    snapshots.add_system(solver.state)
    snapshots.add_task("b - F*z", name="b_full")
    # CFL
    CFL = flow_tools.CFL(solver, initial_dt=dt, cadence=1, safety=1,
                         max_change=1.5, min_change=0.5, max_dt=0.125, threshold=0.05)
    CFL.add_velocities(('u', 'w'))

    # Flow properties
    flow = flow_tools.GlobalFlowProperty(solver, cadence=10)
    flow.add_property("sqrt(u*u + w*w) / R", name='Re')

    # Main loop
    try:
        logger.info('Starting loop')
        start_time = time.time()
        while solver.ok:
            dt = CFL.compute_dt()
            solver.step(dt)
            if (solver.iteration) % report_cadence == 0:
                logger.info('Iteration={:d}, Time={:.2e}, dt={:.2e}, Re={:.2e}'.format(solver.iteration, solver.sim_time, dt, flow.max('Re')))
    except:
        logger.error('Exception raised, triggering end of main loop.')
        raise
    finally:
        end_time = time.time()
        logger.info('beginning merge operation')
        post.merge_process_files(snapshots.base_path, cleanup=True)
        logger.info('Iterations: %i' %solver.iteration)
        logger.info('Sim end time: %f' %solver.sim_time)
        logger.info('Run time: %.2f sec' %(end_time-start_time))
        logger.info('Run time: %f cpu-hr' %((end_time-start_time)/60/60*domain.dist.comm_cart.size))

    return case_name

With the function `rayleigh_benard` defined, we can now set our Rayleigh and Prandtl numbers and our grid resolution, and then we can run cases like so:

In [ ]:
Rayleigh = 1e4
Prandtl  = 1
nz = 32
case_name = rayleigh_benard(Rayleigh, Prandtl, nz, run_time=100)

This low-resolution case takes about 15 seconds to run on my laptop.  The simulation outputs some text data to the screen, reporting the evolved sim time, the number of iterations (timesteps), the size of the timesteps (dt), and the peak Reynolds number in the domain (Re).  The timesteps are conducted using a Runge-Kutta second order, two stage implicit/explicit scheme and the timestep size is adaptively calculated using a CFL criteria (though for this low $\mathrm{Ra}$ case we don't see any change in dt; you will as $\mathrm{Ra}$ increases).  At the end of the run, the data is stored in a directory named `case_name` and is ready for plotting.

From the text outputs, we can see that the simulation reaches a steady state.  We see this from the Reynolds number Re saturating at a fixed value, with no visible changes in time.  This is typical for convection just above onset (as this is at these parameters).

Next we need to plot our data.  The approach here is to compute the data once, and the visualize it separately.  This lets you quickly iterate on the visualization as you try to adjust figures to show something you want to understand and is useful technique.  If we weren't in JupyterLab, these steps could be done in parallel (as could the above simulation).

The data is on disk in a directory named `case_name/snapshots` and the block of code below will create visualizations of the data (as `.png` files) in `case_name/frames`, where you can then import and examine the data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from dedalus.extras import plot_tools
import h5py
from IPython.display import Image

def plot_field(case_name):
    """Save plot of specified tasks for given range of analysis writes."""
    data_dir = case_name+'/snapshots'
    output_path = pathlib.Path(case_name+'/frames').absolute()
    # Create output directory if needed
    if not output_path.exists():
        output_path.mkdir()
    # Plot settings
    tasks = ['b_full']
    scale = 2.5
    dpi = 100
    title_func = lambda sim_time: 't = {:.3f}'.format(sim_time)
    savename_func = lambda write: 'snapshot_{:06}.png'.format(write)
    # Layout
    nrows, ncols = 1, 1
    image = plot_tools.Box(4, 1)
    pad = plot_tools.Frame(0.2, 0.2, 0.1, 0.1)
    margin = plot_tools.Frame(0.3, 0.2, 0.1, 0.1)

    # Create multifigure
    mfig = plot_tools.MultiFigure(nrows, ncols, image, pad, margin, scale)
    fig = mfig.figure
    # Plot writes
    for file in os.listdir(data_dir):
        filename = data_dir+'/'+file
        with h5py.File(filename, mode='r') as file:
            for index, time in enumerate(file['scales/sim_time']):
                for n, task in enumerate(tasks):
                    # Build subfigure axes
                    i, j = divmod(n, ncols)
                    axes = mfig.add_axes(i, j, [0, 0, 1, 1])
                    # Call 3D plotting helper, slicing in time
                    dset = file['tasks'][task]
                    plot_tools.plot_bot_3d(dset, 0, index, axes=axes, title=task, even_scale=True)
                # Add time title
                title = title_func(file['scales/sim_time'][index])
                title_height = 1 - 0.5 * mfig.margin.top / mfig.fig.y
                fig.suptitle(title, x=0.48, y=title_height, ha='left')
                # Save figure
                savename = savename_func(file['scales/write_number'][index])
                savepath = output_path.joinpath(savename)
                fig.savefig(str(savepath), dpi=dpi)
                fig.clear()
    plt.close(fig)

Let's run this for the case we just conducted:

In [ ]:
plot_field(case_name)

And let's display a couple of these in-line in our notebook, choosing a snapshot early in the evolution to begin with:

In [ ]:
Image('./'+case_name+'frames/snapshot_000010.png')

Here we can clearly see the linear, unstable temperature gradient (hot on the bottom and cold on the top).  The temperature ranges from $[-0.5, 0.5]$, so $\pm 0.5$ are the hotest and coldest temperatures in the system (based on boundary forcing).

What do things look like later on?  Here's a snapshot from t=100 (snapshots once every $\Delta t=0.25$, so $t=400 \Delta t=100$):

In [ ]:
Image('./'+case_name+'frames/snapshot_000400.png')

Here we can see we're clearly in the nonlinear state, where hot plumes of convection have extended up from the bottom to the top and cold plumes have reached from the top to the bottom.

Neat.  You can run this case at a resolution of $n_z=16$ just fine too, the images are just a bit less snazzy (unless you're a real fan of pixel art).  At $n_z=16$, this takes about 6 seconds to run.  If you increase resolution you capture features better, if you decrease resolution you do a poorer job and eventually you'll start to experience what's called "spectral ringing" and your solution will be dominated by numerical error.

# Participation

1. Conduct convection experiments at Ra={2e3,2e4,2e5,2e6}. The low Ra runs will need to run for long enough to reach a steady-sate; you can estimate this by watching the peak Re; this can be done at lower resolution if desired. Ra=2e3 takes roughly 300 buoyancy times to equilibrate, and is well resolved at nz=16, whereas Ra=2e6 needs at least nz=64 but is relatively equilibrated by t~50.   

2. For each case (each value of Ra), estimate (and report) at what time the system passes through it's transient (when has the Reynolds number peaked).  Make plots of the thermal buoyancy field using `plot_field(case_name)`.  Show plots after the system passes transient (sometime after Reynolds number drops from its peak value).  

3. Which simulations are steady-state (flow field remains the same), and which are dynamically evolving?  If the system is in steady-state, show one plot, but if it is not a steady state, show a couple of plots (2-3) that illustrate dynamics.  The plots are output every `0.25` buoyancy times, so a couple of adjacent frames will capture 1 bouyancy time.

4. Using these plots, describe how the thermal boundary layers change as the Rayleigh number increases.

5. Quantify (or estimate) the scaling of boundary layer thickness (vertical extent) with Ra. 


One of your simulations above is under-resolved.  Let's see what this looks like in practice.

6. Examine your run at Ra=2e6 at $n_z=64$ more closely (or do a run there if you haven't). What unusual thing is occurring at certain times (hint: look at plots just after transient)? Quantify with plots and description. What do you think is happening?

7. What resolution do you need to successfully run at Ra=2e6? Demonstrate that this works by showing a plot of b_full in the new, working resolution and in the old, default, not-working resolution.  I'd suggest running to `run_time=50` so you're not waiting too long (it'll still take a bit).

**To Turn in**:
Send Ben your answers to 1-7 via Canvas, uploading to Asynchronous Lecture 23.  You can do that in this ipynb and upload as an ipynb.  You can also export to PDF if you prefer.  If you do this on RC's JupyterLab, download the notebook after you complete it, and upload that to Canvas.  Please send by midnight, Thursday April 22.